In [99]:
import numpy as np
from scipy.stats import norm
import math

## Option pricing (Black-Scholes) calculator
*Call option:* $C = S_0 \cdot N(d_{1}) - K \cdot e^{rT} \cdot N(d_2)$

*Put option:* $C = K \cdot e^{rT} \cdot N(d_2) - S_0 * N(d_{1})$

$d_1 = \frac{ln(\frac{S_0}{K})+(r\cdot\frac{\sigma^2}{2})\cdot T}{\sigma \cdot \sqrt{T}}$

$d_2 = d_1 - \sigma \cdot \sqrt{T}$

$S_0$ = Current value of underlying asset

$K$ = Strike price of the option

$T$ = Time to experation (years)

$\sigma$ = Volatility of the underlying asset

$r$ = Risk free rate

$N(x)$ = Cumulative distribution function of the standard normal distribution


In [77]:
def getD1(S, K, r, sigma, T):
    nat_sk = np.log(S/K)
    exp_vol = (sigma**2) / 2
    nom = nat_sk + (r+exp_vol)*T
    denom = sigma * np.sqrt(T)
    return nom / denom

In [79]:
def getD2(d1, sigma, T):
    return d1 - sigma * np.sqrt(T)

In [87]:
def getCallOptionPrice(S, K, sigma, r, T):
    d1 = getProbabilityFactorOne(S, K, r, sigma, T)
    d2 = getProbabilityFactorTwo(d1, sigma, T)
    #norm.cdf
    return S * norm.cdf(d1) - K * math.exp(-r*T) * norm.cdf(d2)

In [93]:
def getPutOptionPrice(S, K, sigma, r, T):
    d1 = getProbabilityFactorOne(S, K, r, sigma, T)
    d2 = getProbabilityFactorTwo(d1, sigma, T)
    #norm.cdf
    return K * math.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [103]:
S = 100 # Current value of underlying asset
K = 95 # Strike price
T = 1 # Years to expiry
r = 0.05 # Risk free
sigma = 0.2 # Volaitlity

call_price = getCallOptionPrice(S, K, sigma, r, T)
print(f"Call option price: {call_price:.2f}")
put_price = getPutOptionPrice(S, K, sigma, r, T)
print(f"Put option price: {put_price:.2f}")

Call option price: 13.35
Put option price: 3.71
